In [1]:
import cv2
import numpy as np 
import tensorflow as tf 
import tflite_runtime.interpreter as tflite
import time

tf.__version__

'2.3.1'

In [2]:
# 加载TensorflowLite模型
interpreter = tf.lite.Interpreter(model_path='hand_landmark.tflite')
interpreter.allocate_tensors()

# 输入输出信息参考ModelCard
input_details = interpreter.get_input_details()
print(str(input_details))
output_details = interpreter.get_output_details()
print(str(output_details))

[{'name': 'input_1', 'index': 0, 'shape': array([  1, 224, 224,   3]), 'shape_signature': array([  1, 224, 224,   3]), 'dtype': <class 'numpy.float32'>, 'quantization': (0.0, 0), 'quantization_parameters': {'scales': array([], dtype=float32), 'zero_points': array([], dtype=int32), 'quantized_dimension': 0}, 'sparsity_parameters': {}}]
[{'name': 'Identity', 'index': 167, 'shape': array([ 1, 63]), 'shape_signature': array([ 1, 63]), 'dtype': <class 'numpy.float32'>, 'quantization': (0.0, 0), 'quantization_parameters': {'scales': array([], dtype=float32), 'zero_points': array([], dtype=int32), 'quantized_dimension': 0}, 'sparsity_parameters': {}}, {'name': 'Identity_1', 'index': 166, 'shape': array([1, 1]), 'shape_signature': array([1, 1]), 'dtype': <class 'numpy.float32'>, 'quantization': (0.0, 0), 'quantization_parameters': {'scales': array([], dtype=float32), 'zero_points': array([], dtype=int32), 'quantized_dimension': 0}, 'sparsity_parameters': {}}, {'name': 'Identity_2', 'index': 16

In [9]:
# 使用1张图片测试
img_path='test.jpg'

img=cv2.imread(img_path)
img = cv2.cvtColor(cv2.flip(img, 1), cv2.COLOR_BGR2RGB)
img=cv2.resize(img,(224,224))

img_array=np.array(img)
img_array=img_array.astype('float32')
img_array/=255
img_array=np.expand_dims(img_array,axis=0)

print(img_array.shape)

interpreter.set_tensor(input_details[0]['index'], img_array)
interpreter.invoke()
output_data = interpreter.get_tensor(output_details[0]['index'])

print(output_data)


# 在图片中添加骨骼点
for i in range(21):
    x=int(output_data[0][3*i])
    y=int(output_data[0][3*i+1])
    cor=(x,y)
    cv2.circle(img,cor,1,(0,0,255),3)

# 转换为BGR输出
img = cv2.cvtColor(cv2.flip(img, 1), cv2.COLOR_RGB2BGR)
cv2.imwrite('test_hand.jpeg', img)


(1, 224, 224, 3)
[[162.67245   176.53564     0.        139.4296    182.94305    -4.4875803
  113.151886  176.47939    -9.9828205  89.14227   174.20378   -15.542915
   64.78345   174.71198   -21.195286  110.07095   129.87729   -12.07172
   70.59595   101.829346  -18.34488    45.45308    85.42633   -21.401375
   27.861767   72.24675   -22.982944  123.52904   114.46585   -13.024407
   82.149216   78.13591   -19.039959   56.07536    57.29424   -21.315962
   38.882828   40.55296   -22.771843  138.3533    105.697495  -14.079844
  107.07805    67.42593   -19.647459   84.18525    47.844093  -21.602793
   67.696335   31.73631   -22.687199  153.99312   101.89351   -15.361776
  140.29681    70.92858   -18.958736  127.48488    50.898117  -20.31353
  116.11608    33.926197  -21.367645 ]]


True

In [3]:
def Show_frame(img,cors):
    for i in range(21):
        cor=(int(cors[3*i]),int(cors[3*i+1]))
        cv2.circle(img,cor,1,(0,0,255),3)


In [4]:
cap = cv2.VideoCapture(0)

while cap.isOpened():

  t_start=time.time()

  success, img = cap.read()
  if not success:
    print("Ignoring empty camera frame.")
    continue
  img = cv2.cvtColor(cv2.flip(img, 1), cv2.COLOR_BGR2RGB)

  img=cv2.resize(img,(224,224))
  
  img_array=np.array(img)
  img_array=img_array.astype('float32')
  img_array/=255
  img_array=np.expand_dims(img_array,axis=0)

  #print(img_array.shape)
  
  interpreter.set_tensor(input_details[0]['index'], img_array)
  interpreter.invoke()
  output_data = interpreter.get_tensor(output_details[0]['index'])
  confidence = interpreter.get_tensor(output_details[1]['index'])
  LR=interpreter.get_tensor(output_details[2]['index'])

  Show_frame(img,output_data[0])

  img=cv2.resize(img,(640,480))
  img = cv2.cvtColor(cv2.flip(img, 1), cv2.COLOR_RGB2BGR)
  cv2.imshow('MediaPipe Hands', img)

  if cv2.waitKey(100) & 0xFF == 27: # 按下 esc 退出
    break

  dt=time.time()-t_start
  print('FPS: ',1/dt,'confidence:',confidence,'L or R:',LR)
cap.release()

FPS:  1.7873741439315716 confidence: [[0.0059123]] L or R: [[0.551375]]
FPS:  5.789047705804086 confidence: [[0.00527129]] L or R: [[0.5611033]]
FPS:  5.859652358080677 confidence: [[0.00564188]] L or R: [[0.54983354]]
FPS:  5.873511598434404 confidence: [[0.00595191]] L or R: [[0.5131255]]
FPS:  5.764740323731169 confidence: [[0.00535682]] L or R: [[0.54732686]]
FPS:  5.743568395409312 confidence: [[0.00708261]] L or R: [[0.5540036]]
FPS:  5.7919335840135435 confidence: [[0.00752142]] L or R: [[0.536655]]
FPS:  5.3172551057922695 confidence: [[0.00937289]] L or R: [[0.57639635]]
FPS:  5.795799248286536 confidence: [[0.01075774]] L or R: [[0.5200723]]
FPS:  6.412700936608456 confidence: [[0.00964698]] L or R: [[0.5522891]]
FPS:  6.464451177973755 confidence: [[0.00878298]] L or R: [[0.6684588]]
FPS:  6.335050651055236 confidence: [[0.00836316]] L or R: [[0.6705274]]
FPS:  5.823230422993568 confidence: [[0.01057774]] L or R: [[0.68055004]]
FPS:  6.465367932002947 confidence: [[0.0170682

In [ ]:
cap = cv2.VideoCapture(0)

while cap.isOpened():

  t_start=time.time()

  success, img = cap.read()
  if not success:
    print("Ignoring empty camera frame.")
    continue

  print(img.shape)
  cv2.imshow('MediaPipe Hands', img)

  if cv2.waitKey(100) & 0xFF == 27: # 按下 esc 退出
    break

  dt=time.time()-t_start
cap.release()